## Predict next words in a sentence using LSTM Recurrent Neural Network (RNN)

In [88]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

In [89]:
#source text
data = open('C:\\Users\\HP.DESKTOP-85IK0UN\\Machine_Learning_Bootcamp\\MinorProject\\word_prediction_model\\story.txt').read()

In [90]:
#integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded_data = tokenizer.texts_to_sequences([data])[0]
encoded_data

[12,
 11,
 9,
 1,
 348,
 586,
 8,
 6,
 587,
 7,
 349,
 74,
 47,
 2,
 246,
 19,
 198,
 350,
 29,
 351,
 21,
 5,
 352,
 3,
 588,
 83,
 589,
 9,
 22,
 7,
 70,
 145,
 199,
 590,
 5,
 12,
 11,
 353,
 20,
 86,
 8,
 1,
 133,
 34,
 591,
 354,
 47,
 2,
 76,
 95,
 3,
 592,
 593,
 64,
 12,
 68,
 31,
 594,
 8,
 87,
 595,
 47,
 50,
 247,
 1,
 596,
 164,
 3,
 597,
 1,
 108,
 134,
 1,
 355,
 50,
 31,
 598,
 5,
 599,
 48,
 356,
 6,
 600,
 601,
 146,
 51,
 31,
 602,
 3,
 603,
 59,
 35,
 604,
 357,
 1,
 200,
 358,
 146,
 201,
 31,
 605,
 121,
 12,
 606,
 3,
 607,
 83,
 7,
 202,
 608,
 27,
 47,
 25,
 165,
 11,
 609,
 610,
 2,
 68,
 611,
 9,
 25,
 359,
 3,
 360,
 96,
 612,
 23,
 50,
 613,
 3,
 614,
 1,
 349,
 3,
 87,
 361,
 615,
 134,
 203,
 6,
 248,
 23,
 3,
 1,
 616,
 617,
 38,
 33,
 618,
 24,
 1,
 86,
 9,
 619,
 620,
 249,
 3,
 6,
 621,
 88,
 351,
 21,
 11,
 622,
 83,
 623,
 250,
 9,
 251,
 624,
 625,
 250,
 47,
 362,
 3,
 626,
 48,
 627,
 8,
 109,
 146,
 31,
 1,
 252,
 7,
 22,
 7,
 25,
 253,
 628,
 89

In [91]:
#determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1 #0 is reserved for padding that's why we added 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 1431


### Next we need to create sequences  of words to fit the model with one word as input and other as output

In [92]:
#create word -> word sequences
sequences = list()

for i in range(1, len(encoded_data)):
    sequence = encoded_data[i-1:i+1]
    sequences.append(sequence)
    
max_sequence_len = max([len(x) for x in sequences])
    
print('Total Sequences: %d' % len(sequences))
#split into X and y elements

Total Sequences: 6525


In [93]:
sequences #Running this piece shows that we have a total of 34 input-output pairs to train the network

[[12, 11],
 [11, 9],
 [9, 1],
 [1, 348],
 [348, 586],
 [586, 8],
 [8, 6],
 [6, 587],
 [587, 7],
 [7, 349],
 [349, 74],
 [74, 47],
 [47, 2],
 [2, 246],
 [246, 19],
 [19, 198],
 [198, 350],
 [350, 29],
 [29, 351],
 [351, 21],
 [21, 5],
 [5, 352],
 [352, 3],
 [3, 588],
 [588, 83],
 [83, 589],
 [589, 9],
 [9, 22],
 [22, 7],
 [7, 70],
 [70, 145],
 [145, 199],
 [199, 590],
 [590, 5],
 [5, 12],
 [12, 11],
 [11, 353],
 [353, 20],
 [20, 86],
 [86, 8],
 [8, 1],
 [1, 133],
 [133, 34],
 [34, 591],
 [591, 354],
 [354, 47],
 [47, 2],
 [2, 76],
 [76, 95],
 [95, 3],
 [3, 592],
 [592, 593],
 [593, 64],
 [64, 12],
 [12, 68],
 [68, 31],
 [31, 594],
 [594, 8],
 [8, 87],
 [87, 595],
 [595, 47],
 [47, 50],
 [50, 247],
 [247, 1],
 [1, 596],
 [596, 164],
 [164, 3],
 [3, 597],
 [597, 1],
 [1, 108],
 [108, 134],
 [134, 1],
 [1, 355],
 [355, 50],
 [50, 31],
 [31, 598],
 [598, 5],
 [5, 599],
 [599, 48],
 [48, 356],
 [356, 6],
 [6, 600],
 [600, 601],
 [601, 146],
 [146, 51],
 [51, 31],
 [31, 602],
 [602, 3],
 [3, 

### We can then split the sequences into input(X) and output elements(y)

In [94]:
sequences = array(sequences)
X, y = sequences[:, 0], sequences[:, 1]

In [95]:
X[:5]

array([ 12,  11,   9,   1, 348])

In [96]:
y[:5]

array([ 11,   9,   1, 348, 586])

In [97]:
#one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)

#define model
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [98]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = 1)) # Embedding Layer
model.add(Bidirectional(LSTM(150, return_sequences=True)))  # Bidirectional LSTM Layer
model.add(Dropout(0.2)) # Dropout Layer
model.add(LSTM(100)) # LSTM Layer
model.add(Dense(vocab_size/2, activation='relu'))  # A Dense Layer including regularizers
model.add(Dense(vocab_size, activation='softmax'))  # A Dense Layer
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1, 10)             14310     
_________________________________________________________________
bidirectional (Bidirectional (None, 1, 300)            193200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 300)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_5 (Dense)              (None, 715)               72215     
_________________________________________________________________
dense_6 (Dense)              (None, 1431)              1024596   
Total params: 1,464,721
Trainable params: 1,464,721
Non-trainable params: 0
____________________________________________

In [99]:
#compile network
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # loss function and an optimizer

In [100]:
#fit network
model.fit(X, y, epochs = 250)

Epoch 1/250
204/204 [==============================] - 3s 14ms/step - loss: 6.2962 - accuracy: 0.0518
Epoch 2/250
204/204 [==============================] - 3s 14ms/step - loss: 5.7826 - accuracy: 0.0562 0s - loss: 5.7906 - accuracy: 0.
Epoch 3/250
204/204 [==============================] - 3s 14ms/step - loss: 5.5859 - accuracy: 0.0581
Epoch 4/250
204/204 [==============================] - 3s 14ms/step - loss: 5.4993 - accuracy: 0.0573
Epoch 5/250
204/204 [==============================] - 3s 13ms/step - loss: 5.4379 - accuracy: 0.0585
Epoch 6/250
204/204 [==============================] - 3s 13ms/step - loss: 5.3817 - accuracy: 0.0582
Epoch 7/250
204/204 [==============================] - 3s 13ms/step - loss: 5.3334 - accuracy: 0.0582
Epoch 8/250
204/204 [==============================] - 3s 13ms/step - loss: 5.2821 - accuracy: 0.0599
Epoch 9/250
204/204 [==============================] - 3s 13ms/step - loss: 5.2312 - accuracy: 0.0628
Epoch 10/250
204/204 [===========================

204/204 [==============================] - 3s 14ms/step - loss: 3.1880 - accuracy: 0.2379
Epoch 80/250
204/204 [==============================] - 3s 14ms/step - loss: 3.1662 - accuracy: 0.2349
Epoch 81/250
204/204 [==============================] - 3s 13ms/step - loss: 3.1614 - accuracy: 0.2408
Epoch 82/250
204/204 [==============================] - 3s 13ms/step - loss: 3.1606 - accuracy: 0.2460
Epoch 83/250
204/204 [==============================] - 3s 13ms/step - loss: 3.1470 - accuracy: 0.2455
Epoch 84/250
204/204 [==============================] - 3s 13ms/step - loss: 3.1327 - accuracy: 0.2503
Epoch 85/250
204/204 [==============================] - 3s 14ms/step - loss: 3.1211 - accuracy: 0.2449
Epoch 86/250
204/204 [==============================] - 3s 14ms/step - loss: 3.1228 - accuracy: 0.2452
Epoch 87/250
204/204 [==============================] - 3s 14ms/step - loss: 3.1056 - accuracy: 0.2457
Epoch 88/250
204/204 [==============================] - 3s 14ms/step - loss: 3.0966 - 

204/204 [==============================] - 3s 14ms/step - loss: 2.7515 - accuracy: 0.2734
Epoch 158/250
204/204 [==============================] - 3s 14ms/step - loss: 2.7566 - accuracy: 0.2733
Epoch 159/250
204/204 [==============================] - 3s 13ms/step - loss: 2.7504 - accuracy: 0.2705
Epoch 160/250
204/204 [==============================] - 3s 13ms/step - loss: 2.7473 - accuracy: 0.2713
Epoch 161/250
204/204 [==============================] - 3s 13ms/step - loss: 2.7450 - accuracy: 0.2759 1s - loss: 2.7084 - accu - E
Epoch 162/250
204/204 [==============================] - 3s 13ms/step - loss: 2.7384 - accuracy: 0.2785
Epoch 163/250
204/204 [==============================] - 3s 13ms/step - loss: 2.7364 - accuracy: 0.2752
Epoch 164/250
204/204 [==============================] - 3s 13ms/step - loss: 2.7348 - accuracy: 0.2783
Epoch 165/250
204/204 [==============================] - 3s 16ms/step - loss: 2.7357 - accuracy: 0.2731
Epoch 166/250
204/204 [==========================

204/204 [==============================] - 3s 13ms/step - loss: 2.6114 - accuracy: 0.2794
Epoch 234/250
204/204 [==============================] - 3s 14ms/step - loss: 2.6067 - accuracy: 0.2737
Epoch 235/250
204/204 [==============================] - 3s 14ms/step - loss: 2.6055 - accuracy: 0.2769
Epoch 236/250
204/204 [==============================] - 3s 14ms/step - loss: 2.6032 - accuracy: 0.2841
Epoch 237/250
204/204 [==============================] - 3s 14ms/step - loss: 2.6014 - accuracy: 0.2766
Epoch 238/250
204/204 [==============================] - 3s 15ms/step - loss: 2.6052 - accuracy: 0.2760
Epoch 239/250
204/204 [==============================] - 3s 15ms/step - loss: 2.6062 - accuracy: 0.2794
Epoch 240/250
204/204 [==============================] - 3s 14ms/step - loss: 2.6046 - accuracy: 0.2821
Epoch 241/250
204/204 [==============================] - 3s 15ms/step - loss: 2.5929 - accuracy: 0.2820
Epoch 242/250
204/204 [==============================] - 3s 14ms/step - loss: 

In [101]:
# generate a sequence from the model
def generate_seq(model, tokenizer, enter_text, n_pred):
    in_text, result = enter_text, enter_text  
    # generate a fix number of words
    for i in range(n_pred):
        
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = array(encoded)
        
        # predict a word in the vocabulary
        yhat = model.predict_classes(encoded)
        
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text, result = out_word, result + ' ' + out_word
    return result

In [102]:
# evaluate
print(generate_seq(model, tokenizer, 'there',10))

there is a man who was a man who was a


In [103]:
# evaluate
print(generate_seq(model, tokenizer, 'english', 6))

english charters researches here and i have


In [104]:
# evaluate
print(generate_seq(model, tokenizer, 'then', 6))

then it is a man who was


In [105]:
# evaluate
print(generate_seq(model, tokenizer, 'what', 6))

what could not a man who was


In [106]:
# evaluate
print(generate_seq(model, tokenizer, 'if', 6))

if you can you can you can


In [107]:
# evaluate
print(generate_seq(model, tokenizer, 'busy', 6))

busy just as we are aware that


In [108]:
# evaluate
print(generate_seq(model, tokenizer, 'right', 6))

right path said holmes that he was
